## Exploring EMIT data and inspecting spectra for alien species
----

### Overview 
----

In this notebook,  we will start analysing satellite imaging spectroscopy data. We will open data from [The Earth Surface Mineral Dust Source Investigation (EMIT)](https://earth.jpl.nasa.gov/emit/) stored remotely on AWS and explore it. We will then extract some spectra for invasive plant species and visualize these.

- 
### Learning Objectives
1. Understand how to access data direclty from AWS s3 storage
2. Load, mask, orthorectify, and visualize EMIT data
3. Extract spectra at locations with and without aline plant and inspect these

----

#### Load libraries

In [ ]:
#imports
import xarray as xr
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
import pandas as pd
import numpy as np
import math
import geopandas as gpd
import panel as pn
import xvec
#our modules
import emit_tools #from https://github.com/nasa/EMIT-Data-Resources/blob/main/python/modules/emit_tools.py

# Function to adjust gamma across all bands - adjust brightness
def gamma_adjust(rgb_ds, bright=0.2, white_background=False):
    array = rgb_ds.reflectance.data
    gamma = math.log(bright)/math.log(np.nanmean(array)) # Create exponent for gamma scaling - can be adjusted by changing 0.2 
    scaled = np.power(array,gamma).clip(0,1) # Apply scaling and clip to 0-1 range
    if white_background == True:
        scaled = np.nan_to_num(scaled, nan = 1) # Assign NA's to 1 so they appear white in plots
    rgb_ds.reflectance.data = scaled
    return rgb_ds

#### Authenticate with NASA Earthdata portal
Earthaccess is a python library to search, download or stream NASA Earth science data. You will also need an account on NASA's Earthdata data portal
https://search.earthdata.nasa.gov/

In [ ]:
import earthaccess
auth = earthaccess.login(persist=True)

##### Accessing data on AWS S3

The emit files are stored on Amazon S3 object storage. We will access these directly without needing to download them as we can stream s3 files. Also, because we are using xarray with dask and our data is stored in a chunked file format `netcdf`, we only need to stream the chunks we are currently working on and not the entire file at once.
  
The best way to access the data is on a virtual machine located in `us-west-2`. This is where the data is located and this mean that will get the fastest reading and writing of data possible, and all read/writes will be free. If your machine is not in `us-west-2` or you are on a local machine, I do not recommend using S3, as it might be slower and there will be costs involved.

##### Finding data

earthaccess has really convineint methods for searching and loading nasa data over S3 using `earthaccess.search_data()`. Here is an example:
```
results = earthaccess.search_data(
    short_name='EMITL2ARFL',
    point=(19.03711,-33.94747),
    temporal=('2023-01-18','2023-01-24'),
    count=2
)


you can then load the results into an xarray
```
files = earthaccess.open(results)
ds = xr.open_dataset(files.f)


if you do not want to stream the data from S3 you can download the data directly using:
```
files = earthaccess.download(results, "./local_folder")


for more info on using earthaccess see the [docs](https://nsidc.github.io/earthaccess/)

#### Open the datset

If you are in us-west-2: Authenticate with s3 and stream data

In [ ]:
results = earthaccess.search_data(
    short_name='EMITL2ARFL',
    point=(19.03711,-33.94747),
    temporal=('2023-01-18','2023-01-24'),
    count=1
)
file_handlers = earthaccess.open(results)
file_handlers

Inspect the raw data

In [ ]:
xr.open_dataset(file_handlers[0].f,engine='h5netcdf',chunks='auto')

If you are not in us-west-2: Download locally

#### Load the mask
The EMIT data is composed of 3 files, the actual data, a mask that tells us about various quality issues with the data, and the reflectance uncertainties (we will ignore the uncertainty for now). We can selecting which quality filter (mask) to use. Here we used the aggregate filter - the sum of all filters to be very strict about what data we include. The `quality_mask` function provided by the EMIT team reads and processes this file for us.

In [ ]:
flags=[7]
mask = emit_tools.quality_mask(file_handlers[2].f,flags)

#### Load the data
As you saw the raw data is the unprojected, unmasked focal plane array and does not include much metadata for us to tell what is going on.  The `emit_array` function provided by the EMIT team reads and processes this file for us. It can orthorectify the data and apply the mask too if we want.

> Warning: in order to orthorectify the data, the entire array must be loaded into memory. This means you need a decent amount of RAM or the operation will fail. I found that 16GB is the minimum needed.


In [ ]:
ds = emit_tools.emit_xarray(file_handlers[0].f, ortho=True, qmask=mask)
ds

we will chunk the data so that we dont can apply any analysis to chunks rather than the entire array

In [ ]:
ds = ds.chunk({'latitude':100,'longitude':100,'wavelengths':10})

The first thing we do is drop the bad bands (those in the atmospheric water absorption zones) and replace -9999 which is used to code no data with nan

In [ ]:
ds = ds.where(ds.good_wavelengths.compute()==1,drop=True)
ds = ds.where(ds.reflectance != -9999)

#### Quick plot

In [ ]:
#select rgb
rgb = ds.sel(wavelengths=[650, 560, 470], method='nearest')
#make colors nicer
rgb = gamma_adjust(rgb)

plot natural colors

In [ ]:
rgb.hvplot.rgb(x='longitude', y='latitude', bands='wavelengths', frame_width=400,rasterize=True, data_aspect=1)

plot a single wavelength

In [ ]:
ds.sel(wavelengths=850, method='nearest').hvplot(cmap='Magma', geo=True, 
                                                 tiles='ESRI', rasterize=True, alpha=0.8, 
                                                 frame_height=600,clim =(0,0.7)
                                                ).opts(title=f"Reflectance at 850nm (Orthorectified)")

We can make nice interactive plots in python. Here we can use a slider to select the wavelength we want to view

In [ ]:
def plot_reflectance(wavelength):
    return ds.sel(wavelengths=wavelength, method='nearest').hvplot(
        cmap='Magma', geo=True, tiles='ESRI', rasterize=True, alpha=0.8,
        frame_height=600, clim=(0, 0.7)
    ).opts(title=f"Reflectance at {wavelength}nm (Orthorectified)")

wavelength_slider = pn.widgets.IntSlider(name='Wavelength (nm)', start=int(min(ds.wavelengths.values)), end=int(max(ds.wavelengths.values)), step=5, value=850)

interactive_plot = pn.bind(plot_reflectance, wavelength=wavelength_slider)

pn.Column(wavelength_slider, interactive_plot)


#### Extract points

We will load a small dataset of points for Fynbos and Pine covered sites across the EMIT image we are working with.

In [ ]:
#load points
gdf = gpd.read_file("shared/users/gmoncrieff/data/emit_unmix.gpkg")
#explore the data
gdf[['Category','geometry']].explore('Category')

The xvec package will extract data at a point or area within an xarray and return the data as an xarray

In [ ]:
extracted = ds.xvec.extract_points(gdf['geometry'], x_coords="longitude", y_coords="latitude",index=True)
extracted


Lets turn our xarray into a dataframe, and join it with the original dataset

In [ ]:
extracted = (extracted
             .to_dataframe() #turn into a dataframe
             .reset_index() #convert the row names to a column
             .merge( #merge with the original dataset
                 gdf.reset_index(),on='index')
            )
#select the columns we want
extracted = extracted[['wavelengths','reflectance','geometry_x','Category','index']]
extracted

Finally lets select a class and plot the spectra for all points of that class

In [ ]:
extracted_plot = extracted.query('Category == "FYNBOS"')
extracted_plot.hvplot.line(y='reflectance',x='wavelengths',by='index',
                                    color='green',ylim=(0,0.5),alpha=0.5,legend=False)

### credits

This lesson has borrowed from:    

[the EMIT-Data-Resources repository by LPDAAC/ the EMIT team](https://github.com/nasa/EMIT-Data-Resources) 
